In [1]:
%pip install pandas requests numpy lxml beautifulsoup4 html5lib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd
import numpy as np
import io


In [3]:
url_wiki = "https://pt.wikipedia.org/wiki/Lista_dos_indicados_ao_Oscar_de_melhor_filme_internacional"
headers = {"User-Agent": "Mozilla/5.0"}

resp_wiki = requests.get(url_wiki, headers=headers)
resp_wiki.raise_for_status()

tables_wiki = pd.read_html(io.StringIO(resp_wiki.text))

decade_tables = [df for df in tables_wiki if 'Filme' in df.columns]

paises = pd.concat(decade_tables, ignore_index=True)
paises

,Ano,Filme,Direção,País concorrente,Língua(s),Ref.
0,1948 (20.ª),Sciuscià[nota 3],Vittorio De Sica,Itália,Italiano,[10]
1,1949 (21.ª),Monsieur Vincent,Maurice Cloche,França,Francês,[11]
2,1950 (22.ª),Ladri di biciclette,Vittorio De Sica,Itália,Italiano,[12]
3,1951 (23.ª),Au-delà des grilles Le mura di Malapaga,René Clément,França & Itália,Francês e Italiano,[13]
4,1952 (24.ª),Rashōmon 羅生門,Akira Kurosawa,Japão,Japonês,[14]
...,...,...,...,...,...,...
348,2025 (97.ª),Ainda Estou Aqui,Walter Salles,Brasil,Português,[86]
349,2025 (97.ª),Dāne-ye anjīr-e ma'ābed,Mohammad Rasoulof,Alemanha,Persa,[86]
350,2025 (97.ª),Emilia Pérez,Jacques Audiard,França,Espanhol,[86]
351,2025 (97.ª),Pigen med nålen,Magnus von Horn,Dinamarca,Dinamarquês,[86]


In [4]:
url_paises = "https://servicodados.ibge.gov.br/api/v1/paises"
resp = requests.get(url_paises, timeout=20)
resp.raise_for_status()
paises_data = resp.json()

len(paises_data), paises_data[0]

(193,
 {'id': {'M49': 20, 'ISO-3166-1-ALPHA-2': 'AD', 'ISO-3166-1-ALPHA-3': 'AND'},
  'nome': {'abreviado': 'Andorra',
   'abreviado-EN': 'Andorra',
   'abreviado-ES': 'Andorra'},
  'area': {'total': '468',
   'unidade': {'nome': 'quilômetros quadrados',
    'símbolo': 'km2',
    'multiplicador': 1}},
  'localizacao': {'regiao': {'id': {'M49': 150}, 'nome': 'Europa'},
   'sub-regiao': {'id': {'M49': 39},
    'nome': 'Europa meridional  (Sul da Europa)'},
   'regiao-intermediaria': None},
  'linguas': [{'id': {'ISO-639-1': 'ca', 'ISO-639-2': 'cat'},
    'nome': 'catalão'}],
  'governo': {'capital': {'nome': 'Andorra-a-Velha'}},
  'unidades-monetarias': [{'id': {'ISO-4217-ALPHA': 'EUR',
     'ISO-4217-NUMERICO': '978'},
    'nome': 'Euro'}],
  'historico': 'O Principado de Andorra é um dos menores Estados da Europa, situado no alto dos Pireneus, entre as fronteiras francesa e espanhola. O primeiro documento que menciona Andorra é a Ata de Consagração da Catedral de Santa María de Urgell,

In [5]:
# Vamos transformar essa resposta num DataFrame organizado, com os campos mais úteis para merge:

registros = []

for pais in paises_data:
    pid = pais.get("id", {})
    nome = pais.get("nome", {})
    loc = pais.get("localizacao", {})
    regiao = (loc.get("regiao") or {}).get("nome")
    sub_regiao = (loc.get("sub-regiao") or {}).get("nome")
    regiao_intermediaria = (loc.get("regiao-intermediaria") or {}).get("nome")

    registros.append({
        "iso2": pid.get("ISO-3166-1-ALPHA-2"),
        "iso3": pid.get("ISO-3166-1-ALPHA-3"),
        "nome_ibge_pt": nome.get("abreviado"),
        "nome_ibge_en": nome.get("abreviado-EN"),
        "nome_ibge_es": nome.get("abreviado-ES"),
        "regiao": regiao,
        "sub_regiao": sub_regiao,
        "regiao_intermediaria": regiao_intermediaria,
    })

ibge_paises_df = pd.DataFrame(registros)
ibge_paises_df

,iso2,iso3,nome_ibge_pt,nome_ibge_en,nome_ibge_es,regiao,sub_regiao,regiao_intermediaria
0,AD,AND,Andorra,Andorra,Andorra,Europa,Europa meridional (Sul da Europa),NaN
1,AE,ARE,Emirados Árabes Unidos,United Arab Emirates,Emiratos Árabes Unidos,Ásia,Ásia ocidental (Oeste da Ásia),NaN
2,AF,AFG,Afeganistão,Afghanistan,Afganistán,Ásia,Ásia meridional (Sul da Ásia),NaN
3,AG,ATG,Antígua e Barbuda,Antigua and Barbuda,Antigua y Barbuda,América,América Latina e Caribe,Caribe
4,AL,ALB,Albânia,Albania,Albania,Europa,Europa meridional (Sul da Europa),NaN
...,...,...,...,...,...,...,...,...
188,WS,WSM,Samoa,Samoa,Samoa,Oceania,Polinésia,NaN
189,YE,YEM,Iêmen,Yemen,Yemen,Ásia,Ásia ocidental (Oeste da Ásia),NaN
190,ZA,ZAF,África do Sul,South Africa,Sudáfrica,África,África subsaariana,África meridional (Sul da África)
191,ZM,ZMB,Zâmbia,Zambia,Zambia,África,África subsaariana,África oriental (Leste da África)


In [6]:
# Precisamos alinhar com os nomes do IBGE (nome_ibge_pt).
# Em muitos casos bate direto;
# em outros talvez seja necessário um pequeno dicionário manual para ajustar grafias
# (por exemplo, “Coreia do Sul” vs “República da Coreia”, “Estados Unidos” vs “Estados Unidos da América”).

sorted(paises["País concorrente"].unique())[:30]

['Alemanha',
 'Alemanha Ocidental',
 'Alemanha Oriental',
 'Argentina',
 'Argélia',
 'Austrália',
 'Brasil',
 'Butão',
 'Bélgica',
 'Bósnia e Herzegovina',
 'Camboja',
 'Canadá',
 'Cazaquistão',
 'Checoslováquia',
 'Chile',
 'China',
 'Chéquia',
 'Colômbia',
 'Coreia do Sul',
 'Costa do Marfim',
 'Cuba',
 'Dinamarca',
 'Espanha',
 'Estónia',
 'Finlândia',
 'França',
 'França & Itália',
 'Geórgia',
 'Grécia',
 'Hong Kong']

In [7]:
# faz um merge entre paises e ibge_paises_df
vencedores_merged = paises.merge(
    ibge_paises_df,
    left_on="País concorrente",
    right_on="nome_ibge_pt",
    how="left"
)

vencedores_merged

,Ano,Filme,Direção,País concorrente,Língua(s),Ref.,iso2,iso3,nome_ibge_pt,nome_ibge_en,nome_ibge_es,regiao,sub_regiao,regiao_intermediaria
0,1948 (20.ª),Sciuscià[nota 3],Vittorio De Sica,Itália,Italiano,[10],IT,ITA,Itália,Italy,Italia,Europa,Europa meridional (Sul da Europa),NaN
1,1949 (21.ª),Monsieur Vincent,Maurice Cloche,França,Francês,[11],FR,FRA,França,France,Francia,Europa,Europa ocidental (Oeste da Europa),NaN
2,1950 (22.ª),Ladri di biciclette,Vittorio De Sica,Itália,Italiano,[12],IT,ITA,Itália,Italy,Italia,Europa,Europa meridional (Sul da Europa),NaN
3,1951 (23.ª),Au-delà des grilles Le mura di Malapaga,René Clément,França & Itália,Francês e Italiano,[13],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1952 (24.ª),Rashōmon 羅生門,Akira Kurosawa,Japão,Japonês,[14],JP,JPN,Japão,Japan,Japón,Ásia,Ásia oriental (Leste da Ásia),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,2025 (97.ª),Ainda Estou Aqui,Walter Salles,Brasil,Português,[86],BR,BRA,Brasil,Brazil,Brasil,América,América Latina e Caribe,América do sul
349,2025 (97.ª),Dāne-ye anjīr-e ma'ābed,Mohammad Rasoulof,Alemanha,Persa,[86],DE,DEU,Alemanha,Germany,Alemania,Europa,Europa ocidental (Oeste da Europa),NaN
350,2025 (97.ª),Emilia Pérez,Jacques Audiard,França,Espanhol,[86],FR,FRA,França,France,Francia,Europa,Europa ocidental (Oeste da Europa),NaN
351,2025 (97.ª),Pigen med nålen,Magnus von Horn,Dinamarca,Dinamarquês,[86],DK,DNK,Dinamarca,Denmark,Dinamarca,Europa,Europa setentrional (Norte da Europa),NaN


In [8]:
# Contar os indicados por sub_regiao

contagem_sub_regiao = (
    vencedores_merged
    .groupby("sub_regiao", dropna=True)
    .size()
    .reset_index(name="num_indicados")
    .sort_values("num_indicados", ascending=False)
)
contagem_sub_regiao

,sub_regiao,num_indicados
4,Europa ocidental (Oeste da Europa),79
3,Europa meridional (Sul da Europa),62
6,Europa setentrional (Norte da Europa),43
0,América Latina e Caribe,29
5,Europa oriental (Leste da Europa),22
13,Ásia oriental (Leste da Ásia),20
12,Ásia ocidental (Oeste da Ásia),14
1,América do Norte,7
8,África setentrional (Norte da África),6
11,Ásia meridional (Sul da Ásia),5
